In [ ]:
import pandas as pd
import numpy as np
#import bootstrapped.bootstrap as bts
#import bootstrapped.stats_functions as bs_stats
import os
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pylab as pl
from os import path

# settings for making nice pdfs
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['font.family'] = "sans-serif"

In [ ]:
base_folder_string = 'output_0427_tiling600_TargetedLoading_run'
folders = [foldername for foldername in os.listdir('Data/') if foldername.startswith(base_folder_string)]

base_stochasticity = 1 # use to convert step to time

for m in range(len(folders)):
    data_dir =  folders[m]
    print(data_dir)
    stats_file = [filename for filename in os.listdir('Data/'+data_dir) if filename.startswith("stats_df")]

    sep_list = list(set([stats_file[i].split('stats_df_sep')[1].split('_proc')[0] for i in range(len(stats_file))]))
    sep_numerical_list = [float(a) for a in sep_list]
    sep_list_sorted = [x for _,x in sorted(zip(sep_numerical_list,sep_list))]
    proc_list = list(set([stats_file[i].split('_proc')[1].split('_superportion')[0] for i in range(len(stats_file))]))
    proc_numerical_list = [float(a) for a in proc_list]
    proc_list_sorted = [x for _,x in sorted(zip(proc_numerical_list,proc_list))]
    proc_numerical_list_sorted = [int(a) for a in proc_list_sorted]
    superportion_list = list(set([stats_file[i].split('superportion')[1].split('_superproc')[0] for i in range(len(stats_file))]))
    superportion_numerical_list = [float(a) for a in superportion_list]
    superportion_list_sorted = [x for _,x in sorted(zip(superportion_numerical_list,superportion_list))]
    superportion_numerical_sorted = [float(a) for a in superportion_list_sorted]
    superproc_list = list(set([stats_file[i].split('superproc')[1].split('_ctcf')[0] for i in range(len(stats_file))]))
    superproc_numerical_list = [float(a) for a in superproc_list]
    superproc_list_sorted = [x for _,x in sorted(zip(superproc_numerical_list,superproc_list))]
    superproc_numerical_sorted = [int(a) for a in superproc_list_sorted]
    ctcf_list = list(set([stats_file[i].split('ctcf')[1].split('_dsb')[0] for i in range(len(stats_file))]))
    ctcf_numerical_list = [float(a) for a in ctcf_list]
    ctcf_list_sorted = [x for _,x in sorted(zip(ctcf_numerical_list,ctcf_list))]
    ctcf_numerical_sorted = [float(a) for a in ctcf_list_sorted]
    dsb_list = list(set([stats_file[i].split('dsb')[1].split('_superloading')[0] for i in range(len(stats_file))]))
    dsb_numerical_list = [float(a) for a in dsb_list]
    dsb_list_sorted = [x for _,x in sorted(zip(dsb_numerical_list,dsb_list))]
    superloading_list = list(set([stats_file[i].split('superloading')[1].split('_bs')[0] for i in range(len(stats_file))]))
    superloading_numerical_list = [float(a) for a in superloading_list]
    superloading_list_sorted = [x for _,x in sorted(zip(superloading_numerical_list,superloading_list))]
    superloading_numerical_sorted = [float(a) for a in superloading_list_sorted]
    superloading_ratio = [np.around(100*superloading/(superloading+1e4),decimals=0) for superloading in superloading_numerical_sorted]
    bs_list = list(set([stats_file[i].split('bs')[1].split('.csv')[0] for i in range(len(stats_file))]))
    bs_numerical_list = [float(a) for a in bs_list]
    bs_list_sorted = [x for _,x in sorted(zip(bs_numerical_list,bs_list))]
    bs_numerical_sorted = [float(a) for a in bs_list_sorted]

    if m == 0:
        ctcf_bs_fp = np.zeros((len(folders),len(proc_list_sorted),len(superloading_list_sorted)))
        ctcf_bs_success =   np.zeros((len(folders),len(proc_list_sorted),len(superloading_list_sorted)))
        ctcf_bs_realtime =   np.zeros((len(folders),len(proc_list_sorted),len(superloading_list_sorted)))
        ctcf_bs_time0 =   np.zeros((len(folders),len(proc_list_sorted),len(superloading_list_sorted)))
        captured = np.zeros((len(folders),len(proc_list_sorted),len(superloading_list_sorted)))
        
    for i in range(len(proc_list_sorted)):
        for j in range(len(superloading_list_sorted)):
            par_combo = 'sep' + sep_list_sorted[0] + '_proc' + proc_list_sorted[i] + '_superportion' + superportion_list_sorted[0] + '_superproc' + str(proc_numerical_list_sorted[i]*20) + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[0] + '_superloading' + superloading_list_sorted[j]+ '_bs' + bs_list_sorted[0]
            filename = 'Data/'+data_dir + '/stats_df_' + par_combo + '.csv'
            if path.exists(filename):
                stats_df = pd.read_csv(filename)
                stats_df = stats_df.drop(columns='Unnamed: 0')
                stats_df_threshold1 = stats_df.loc[stats_df['threshold']==1]
                sep_proc_1st_pass = stats_df_threshold1['first_passage_time'].to_numpy()
                if len(sep_proc_1st_pass)>0:
                    ctcf_bs_fp[m,i,j]=np.mean(sep_proc_1st_pass)*base_stochasticity/60 *2
                else:
                    ctcf_bs_fp[m,i,j]=np.nan


                portion_df = pd.read_csv('Data/'+data_dir + '/restrained_df_' + par_combo + '.csv')
                portion_df = portion_df.drop(columns='Unnamed: 0')
                portion_df_threshold1 = portion_df.loc[portion_df['threshold']==1]
                ctcf_bs_time0[m,i,j] = portion_df_threshold1['restrained proportion time0'].to_numpy()[0]*100
                ctcf_bs_success[m,i,j] = portion_df_threshold1['repaired proportion'].to_numpy()[0] * 100

# Theory prediction from Mathematica Notebook Targetedloading_20211004.nb, Probconstrained
theory = np.asarray([[0.0484416, 0.120263, 0.165071, 0.195041, 0.21633, 0.232175, 0.244405, \
0.254119, 0.262016, 0.26856, 0.274068, 0.278769, 0.282826, 0.286363, \
0.289474, 0.292231, 0.294691, 0.296899, 0.298893, 0.300702, 0.30235, \
0.303859, 0.305244, 0.306521, 0.307702, 0.308797, 0.309815, 0.310764, \
0.311651, 0.312482, 0.313261, 0.313994, 0.314685, 0.315337, 0.315953, \
0.316536, 0.317089, 0.317614, 0.318113, 0.318587, 0.31904, 0.319472, \
0.319884, 0.320278, 0.320656, 0.321017, 0.321363, 0.321696, 0.322015, \
0.322322, 0.322617, 0.322901, 0.323175, 0.323439, 0.323694, 0.323939, \
0.324176, 0.324406, 0.324627, 0.324842, 0.32505, 0.325251, 0.325446, \
0.325634, 0.325818, 0.325995, 0.326168, 0.326336, 0.326498, 0.326657, \
0.326811, 0.32696, 0.327106, 0.327248, 0.327386, 0.327521, 0.327652, \
0.32778, 0.327905, 0.328026, 0.328145, 0.328261, 0.328374, 0.328484, \
0.328592, 0.328697, 0.3288, 0.328901, 0.329, 0.329096, 0.32919, \
0.329283, 0.329373, 0.329461, 0.329548, 0.329633, 0.329716, 0.329797, \
0.329877, 0.329955, 0.330032],[0.113293, 0.214307, 0.275396, 0.31577, 0.344282, 0.365433, 0.381723, \
0.394643, 0.405136, 0.413822, 0.42113, 0.427363, 0.43274, 0.437426, \
0.441546, 0.445197, 0.448453, 0.451376, 0.454015, 0.456408, 0.458588, \
0.460583, 0.462415, 0.464103, 0.465664, 0.467112, 0.468457, 0.469712, \
0.470884, 0.471982, 0.473012, 0.47398, 0.474893, 0.475754, 0.476567, \
0.477338, 0.478068, 0.478761, 0.47942, 0.480047, 0.480645, 0.481215, \
0.48176, 0.48228, 0.482778, 0.483255, 0.483713, 0.484152, 0.484573, \
0.484978, 0.485368, 0.485743, 0.486105, 0.486453, 0.486789, 0.487113, \
0.487426, 0.487729, 0.488022, 0.488305, 0.488579, 0.488844, 0.489101, \
0.48935, 0.489592, 0.489827, 0.490055, 0.490276, 0.490491, 0.4907, \
0.490903, 0.4911, 0.491293, 0.49148, 0.491662, 0.49184, 0.492013, \
0.492182, 0.492346, 0.492507, 0.492663, 0.492816, 0.492965, 0.493111, \
0.493253, 0.493392, 0.493528, 0.493661, 0.493791, 0.493918, 0.494042, \
0.494164, 0.494283, 0.4944, 0.494514, 0.494626, 0.494736, 0.494843, \
0.494948, 0.495051, 0.495153],[0.204531, 0.324671, 0.395544, 0.441935, 0.474547, 0.498679, 0.517237, \
0.531943, 0.543876, 0.553751, 0.562056, 0.569137, 0.575244, 0.580566, \
0.585244, 0.589388, 0.593084, 0.596402, 0.599396, 0.602111, 0.604586, \
0.606849, 0.608928, 0.610843, 0.612614, 0.614256, 0.615783, 0.617206, \
0.618535, 0.61978, 0.620949, 0.622047, 0.623082, 0.624058, 0.624981, \
0.625855, 0.626683, 0.627469, 0.628216, 0.628927, 0.629605, 0.630252, \
0.630869, 0.631459, 0.632024, 0.632565, 0.633084, 0.633581, 0.634059, \
0.634519, 0.634961, 0.635386, 0.635795, 0.63619, 0.636571, 0.636939, \
0.637294, 0.637637, 0.637969, 0.63829, 0.6386, 0.638901, 0.639193, \
0.639475, 0.639749, 0.640015, 0.640274, 0.640524, 0.640768, 0.641005, \
0.641235, 0.641459, 0.641677, 0.641889, 0.642096, 0.642297, 0.642493, \
0.642685, 0.642871, 0.643053, 0.64323, 0.643404, 0.643573, 0.643738, \
0.643899, 0.644057, 0.644211, 0.644362, 0.644509, 0.644653, 0.644794, \
0.644932, 0.645067, 0.645199, 0.645329, 0.645455, 0.64558, 0.645701, \
0.645821, 0.645938, 0.646052]])*100

nlist = np.asarray([1, 101, 201, 301, 401, 501, 601, 701, 801, 901, 1001, 1101, 1201, \
1301, 1401, 1501, 1601, 1701, 1801, 1901, 2001, 2101, 2201, 2301, \
2401, 2501, 2601, 2701, 2801, 2901, 3001, 3101, 3201, 3301, 3401, \
3501, 3601, 3701, 3801, 3901, 4001, 4101, 4201, 4301, 4401, 4501, \
4601, 4701, 4801, 4901, 5001, 5101, 5201, 5301, 5401, 5501, 5601, \
5701, 5801, 5901, 6001, 6101, 6201, 6301, 6401, 6501, 6601, 6701, \
6801, 6901, 7001, 7101, 7201, 7301, 7401, 7501, 7601, 7701, 7801, \
7901, 8001, 8101, 8201, 8301, 8401, 8501, 8601, 8701, 8801, 8901, \
9001, 9101, 9201, 9301, 9401, 9501, 9601, 9701, 9801, 9901, 10001])

start = 2
colors = ['aqua','slateblue','violet','deeppink','mediumorchid']
matplotlib.rcParams.update({'font.size': 22})       
fig, axs = plt.subplots(2,1,figsize=(8, 12))
    
for i in range(len(proc_list_sorted)):
    axs[0].errorbar(superloading_numerical_sorted , np.transpose(np.mean(ctcf_bs_success,axis=0)[i,:]),yerr =stats.sem(ctcf_bs_success,axis=0)[i,:],fmt='s',markersize = 8, ecolor= colors[i],color= colors[i],capsize=10,linewidth=3)

for i in range(len(proc_list_sorted)):
    axs[0].plot(nlist,theory[i],color= colors[i],linewidth=3,linestyle='dashed')
    
axs[0].set_ylabel('Synapsis efficiency (%)')
axs[0].set_xlabel('Targeted loading factor')
axs[0].tick_params(direction='out', length=8, width=2)
axs[0].set_xticks(superloading_numerical_sorted)
axs[0].set_xscale('linear')
for axis in ['top','bottom','left','right']:
    axs[0].spines[axis].set_linewidth(1.5)
axs[0].set_ylim(0,100)
    
for axis in ['top','bottom','left','right']:
    axs[0].spines[axis].set_linewidth(1.5)
    
# Hide the right and top spines
axs[0].spines['right'].set_visible(False)
axs[0].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[0].yaxis.set_ticks_position('left')
axs[0].xaxis.set_ticks_position('bottom')
    
for i in range(len(proc_list_sorted)):
    axs[1].errorbar(superloading_numerical_sorted , np.transpose(np.mean(ctcf_bs_fp,axis=0)[i,:]),yerr =stats.sem(ctcf_bs_fp,axis=0)[i,:],fmt='s',markersize = 8,ecolor= colors[i],color= colors[i],capsize=10,linewidth=3)

axs[1].set_ylabel('Mean synapsis time (min)')
axs[1].set_xlabel('Targeted loading factor')
axs[1].tick_params(direction='out', length=8, width=2)
axs[1].set_xticks(superloading_numerical_sorted[0:])
axs[1].set_xscale('linear')

for axis in ['top','bottom','left','right']:
    axs[1].spines[axis].set_linewidth(1.5)
    
legend = ['processivity='+proc for proc in proc_list_sorted]
lgd = axs[1].legend(legend,loc='upper center', bbox_to_anchor=(0.5, -0.2),
          fancybox=True, shadow=True, ncol=1)    
    
# Hide the right and top spines
axs[1].spines['right'].set_visible(False)
axs[1].spines['top'].set_visible(False)

# Only show ticks on the left and bottom spines
axs[1].yaxis.set_ticks_position('left')
axs[1].xaxis.set_ticks_position('bottom')

plt.savefig('Figures/'+'Targeted_loading.pdf',format='pdf',bbox_extra_artists=(lgd,),bbox_inches='tight')
plt.show()

In [ ]:
base_folder_string = 'output_0427_tiling600_TargetedLoading_run'
folders = [foldername for foldername in os.listdir('Data/') if base_folder_string in foldername]
print(folders)

base_stochasticity = 1 # use to convert step to time
for m in range(len(folders)):
    data_dir = folders[m]
    # load DSB coordinates
    with open('Data/'+data_dir+'/DSB_boundary_coordinates.npy',"rb") as f:
        DSB_coordinates = np.load(f)
        boundary_coordinates = np.load(f)
    print(data_dir)
    stats_file = [filename for filename in os.listdir('Data/'+data_dir) if filename.startswith("stats_df")]

    sep_proc_list = ['sep125_proc250']

    superportion_list = list(set([stats_file[i].split('superportion')[1].split('_superproc')[0] for i in range(len(stats_file))]))
    superportion_numerical_list = [float(a) for a in superportion_list]
    superportion_list_sorted = [x for _,x in sorted(zip(superportion_numerical_list,superportion_list))]
    superportion_numerical_sorted = [float(a) for a in superportion_list_sorted]
    superproc_list = list(set([stats_file[i].split('superproc')[1].split('_ctcf')[0] for i in range(len(stats_file))]))
    superproc_numerical_list = [float(a) for a in superproc_list]
    superproc_list_sorted = [x for _,x in sorted(zip(superproc_numerical_list,superproc_list))]
    superproc_numerical_sorted = [int(a) for a in superproc_list_sorted]
    ctcf_list = list(set([stats_file[i].split('ctcf')[1].split('_dsb')[0] for i in range(len(stats_file))]))
    ctcf_numerical_list = [float(a) for a in ctcf_list]
    ctcf_list_sorted = [x for _,x in sorted(zip(ctcf_numerical_list,ctcf_list))]
    ctcf_numerical_sorted = [float(a) for a in ctcf_list_sorted]
    dsb_list = list(set([stats_file[i].split('dsb')[1].split('_superloading')[0] for i in range(len(stats_file))]))
    dsb_numerical_list = [float(a) for a in dsb_list]
    dsb_list_sorted = [x for _,x in sorted(zip(dsb_numerical_list,dsb_list))]
    superloading_list = ['1','250','500','750']
    print(superloading_list)
    superloading_numerical_list = [float(a) for a in superloading_list]
    superloading_list_sorted = [x for _,x in sorted(zip(superloading_numerical_list,superloading_list))]
    bs_list = list(set([stats_file[i].split('bs')[1].split('.csv')[0] for i in range(len(stats_file))]))
    bs_numerical_list = [float(a) for a in bs_list]
    bs_list_sorted = [x for _,x in sorted(zip(bs_numerical_list,bs_list))]
    bs_numerical_sorted = [float(a) for a in bs_list_sorted]

    if m == 0:
        ctcf_bs_fp = np.zeros((len(folders),len(superloading_list),len(sep_proc_list)))
        ctcf_bs_success =  np.zeros((len(folders),len(superloading_list),len(sep_proc_list)))
        ctcf_bs_realtime =  np.zeros((len(folders),len(superloading_list),len(sep_proc_list)))
        ctcf_bs_time0 =  np.zeros((len(folders),len(superloading_list),len(sep_proc_list)))
        enrichment =  np.zeros((6, len(folders),len(superloading_list),len(sep_proc_list)))

    for i in range(len(superloading_list)):
        for j in range(len(sep_proc_list)):
            proc = float(sep_proc_list[j].split('_proc')[1])
            if proc == 62.5:
                par_combo = sep_proc_list[j] + '_superportion' + superportion_list_sorted[0] + '_superproc' + str(proc*20) + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[0]+ '_superloading' + superloading_list_sorted[i]+ '_bs' + bs_list_sorted[0]
            else:
                par_combo = sep_proc_list[j] + '_superportion' + superportion_list_sorted[0] + '_superproc' + str(int(proc*20)) + '_ctcf' + ctcf_list_sorted[0] + '_dsb' + dsb_list_sorted[0]+ '_superloading' + superloading_list_sorted[i]+ '_bs' + bs_list_sorted[0]

            filename = 'Data/'+ data_dir + '/stats_df_' + par_combo + '.csv'
            if path.exists(filename):
                stats_df = pd.read_csv(filename)
                stats_df = stats_df.drop(columns='Unnamed: 0')
                stats_df_threshold1 = stats_df.loc[stats_df['threshold']==1]
                sep_proc_1st_pass = stats_df_threshold1['first_passage_time'].to_numpy()
                if len(sep_proc_1st_pass)>0:
                    ctcf_bs_fp[m,i,j]=np.mean(sep_proc_1st_pass)*base_stochasticity/60 *2
                else:
                    ctcf_bs_fp[m,i,j]=np.nan


                portion_df = pd.read_csv('Data/'+data_dir + '/restrained_df_' + par_combo + '.csv')
                portion_df = portion_df.drop(columns='Unnamed: 0')
                portion_df_threshold1 = portion_df.loc[portion_df['threshold']==1]
                ctcf_bs_time0[m,i,j] = portion_df_threshold1['restrained proportion time0'].to_numpy()[0]
                ctcf_bs_realtime[m,i,j] = portion_df_threshold1['restrained proportion realtime'].to_numpy()[0]
                ctcf_bs_success[m,i,j] = portion_df_threshold1['repaired proportion'].to_numpy()[0] * 100
                
                with open('Data/'+data_dir+'/LEFcoordinates_' + par_combo + '.npy', 'rb') as f:
                    LEF_coordiates=np.load(f)
                    
                LEF_count = np.zeros((7,int(len(DSB_coordinates)/2)))
                for k in range(int(len(DSB_coordinates)/2)-1):
                    index = np.searchsorted(boundary_coordinates,DSB_coordinates[k*2])
                    left_boundary = boundary_coordinates[index-1]
                    right_boundary = boundary_coordinates[index]
                    test_1 = np.where((LEF_coordiates[::2,:]-left_boundary)*(LEF_coordiates[::2,:]-right_boundary)<=0,1,0) 
                    test_2 = np.where((LEF_coordiates[1::2,:]-left_boundary)*(LEF_coordiates[1::2,:]-right_boundary)<=0,1,0) 
                    test_binary = test_1 + test_2
                    LEF_count[:,k] = np.count_nonzero(test_binary,axis=1)
                

                # Normalization 
                LEF_count = np.mean(LEF_count,axis=1)
                LEF_count[1:] = LEF_count[1:]/LEF_count[0]
                enrichment[:,m,i,j]=LEF_count[1:]
            

matplotlib.rcParams.update({'font.size': 22})
fig, axs = plt.subplots(1,1,figsize=(25, 15))

# width of the bars
barWidth = 0.2
r1 = np.arange(6)
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth*2 for x in r1]
r4 = [x + barWidth*3 for x in r1]


bars1 = np.mean(enrichment[:,:,0,0],axis=1)
yer1 = stats.sem(enrichment[:,:,0,0],axis=1,nan_policy='omit')
bars2 = np.mean(enrichment[:,:,1,0],axis=1)
yer2 = stats.sem(enrichment[:,:,1,0],axis=1,nan_policy='omit')
bars3 = np.mean(enrichment[:,:,2,0],axis=1)
yer3 = stats.sem(enrichment[:,:,2,0],axis=1,nan_policy='omit')
bars4 = np.mean(enrichment[:,:,3,0],axis=1)
yer4 = stats.sem(enrichment[:,:,3,0],axis=1,nan_policy='omit')  

# Create blue bars
axs.bar(r1, bars1, width = barWidth, color = 'blue', edgecolor = 'black', yerr=yer1, capsize=7, label='no targeted loading',error_kw={'elinewidth':4,'capsize':10,'capthick':4})

# Create cyan bars
axs.bar(r2, bars2, width = barWidth, color = 'cyan', edgecolor = 'black', yerr=yer2, capsize=7, label='250X targeted loading',error_kw={'elinewidth':4,'capsize':10,'capthick':4})

# Create skyblue bars
axs.bar(r3, bars3, width = barWidth, color = 'steelblue', edgecolor = 'black', yerr=yer3, capsize=7, label='500X targeted loading',error_kw={'elinewidth':4,'capsize':10,'capthick':4})

# Create slateblue bars
axs.bar(r4, bars4, width = barWidth, color = 'mediumslateblue', edgecolor = 'black', yerr=yer3, capsize=7, label='750X targeted loading',error_kw={'elinewidth':4,'capsize':10,'capthick':4})

for i in range(len(r1)):
    axs.scatter(r1[i] + np.random.random(enrichment[:,:,0,0][i,:].size) * barWidth - barWidth / 2, enrichment[:,:,0,0][i,:], s=100,color='blue',edgecolors='k')
for i in range(len(r2)):
    axs.scatter(r2[i] + np.random.random(enrichment[:,:,1,0][i,:].size) * barWidth - barWidth / 2, enrichment[:,:,1,0][i,:], s=100,color='cyan',edgecolors='k')
for i in range(len(r3)):
    axs.scatter(r3[i] + np.random.random(enrichment[:,:,2,0][i,:].size) * barWidth - barWidth / 2, enrichment[:,:,2,0][i,:], s=100,color='steelblue',edgecolors='k')
for i in range(len(r4)):
    axs.scatter(r4[i] + np.random.random(enrichment[:,:,3,0][i,:].size) * barWidth - barWidth / 2, enrichment[:,:,3,0][i,:], s=100,color='mediumslateblue',edgecolors='k')



# Arnould et al. chr20 TAD data
axs.hlines(1.57, xmin=-0.1, xmax=5.5,linestyles='dashed') 

# general layout
axs.set_xticks([r + barWidth*1.5 for r in range(len(bars1))])
axs.set_xticklabels([ '4 min After DSB', '10 min After DSB', '20 min After DSB' , '60 min After DSB', '90 min After DSB', '120 min After DSB'])
axs.set_ylabel('Fold enrichment')
axs.set_title('Sep=125kb, Proc=250kb')
axs.legend()
plt.savefig('Figures/'+'TargetedloadingEnrichment_v2.pdf',format='pdf',bbox_inches='tight')
plt.show()


In [ ]:
bars4[-1]/1.57*100

In [ ]:
bars3[-1]/1.57*100

In [ ]:
bars2[-1]/1.57*100

In [ ]:
bars1[-1]/1.57*100